In [ ]:
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [2]:
digits = load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target,
                                                    train_size=0.75, test_size=0.25)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1347, 64), (450, 64), (1347,), (450,))

In [3]:
tpot = TPOTClassifier(verbosity=2, max_time_mins=5, population_size=40)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

Optimization Progress:  51%|█████▏    | 41/80 [01:14<00:45,  1.16s/pipeline]

Generation 1 - Current best internal CV score: 0.9891007793284805


Optimization Progress:  68%|██████▊   | 81/120 [02:08<00:33,  1.18pipeline/s]

Generation 2 - Current best internal CV score: 0.9898883509262202


Optimization Progress:  96%|█████████▌| 115/120 [00:00<00:02,  2.04pipeline/s]

Generation 3 - Current best internal CV score: 0.9898883509262202


Optimization Progress:  99%|█████████▉| 158/160 [00:00<00:01,  1.48pipeline/s]

Generation 4 - Current best internal CV score: 0.9898883509262202


GP closed prematurely - will use current best pipeline

Best pipeline: KNeighborsClassifier(CombineDFs(XGBClassifier(Binarizer(input_matrix, 0.62), 15, 80, 20.0, 15.0), ZeroCount(RFE(input_matrix, 28.0))), 90, 9)
0.993419911157


In [4]:
tpot.export('tpot_mnist_pipeline.py')

In [ ]:
# %load tpot_mnist_pipeline.py
import numpy as np

from sklearn.ensemble import VotingClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import Binarizer, FunctionTransformer
from sklearn.svm import SVC
from tpot.operators.preprocessors import ZeroCount
from xgboost import XGBClassifier

# NOTE: Make sure that the class is labeled 'class' in the data file
tpot_data = np.recfromcsv('PATH/TO/DATA/FILE', delimiter='COLUMN_SEPARATOR', dtype=np.float64)
features = np.delete(tpot_data.view(np.float64).reshape(tpot_data.size, -1), tpot_data.dtype.names.index('class'), axis=1)
training_features, testing_features, training_classes, testing_classes = \
    train_test_split(features, tpot_data['class'], random_state=42)

exported_pipeline = make_pipeline(
    make_union(
        make_union(VotingClassifier([('branch',
            make_pipeline(
                Binarizer(threshold=0.62),
                XGBClassifier(learning_rate=1.0, max_depth=10, min_child_weight=20, n_estimators=500, subsample=1.0)
            )
        )]), FunctionTransformer(lambda X: X)),
        make_pipeline(
            RFE(estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
              decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
              max_iter=-1, probability=False, random_state=42, shrinking=True,
              tol=0.001, verbose=False), step=0.99),
            ZeroCount()
        )
    ),
    KNeighborsClassifier(n_neighbors=5, weights="distance")
)

exported_pipeline.fit(training_features, training_classes)
results = exported_pipeline.predict(testing_features)
